# Setup

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import nltk
import re

from tqdm import tqdm
tqdm.pandas()

In [2]:
# Baixa recursos necessários do NLTK (executar uma vez)
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/alysson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alysson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df = pd.read_parquet("../data/df_noticias_saude_irregularidade.parquet")

# filtrando as que foram resumidas, ou seja, após validação humana
df = df[~df["human_summary"].isna()].copy()
df.shape

(421, 11)

In [4]:
# filtrando infos úteis
cols_to_use = ["id_folder", "headline", "news_content"]
df = df[cols_to_use].copy()
df

,id_folder,headline,news_content
5818,0a2dfc71-c0c5-46ce-96ce-2e56ae73930c,Ministério da Saúde contraria orientação da CG...,Apesar de a Controladoria Geral da União (CGU)...
5819,8f6d7604-046d-4c99-afc3-d7fd8452582c,Clínica clandestina denunciada por torturas e ...,A Superintendência de Vigilância em Saúde de G...
5820,ddd96e6b-d288-46da-850a-b6f01f1e7958,Diretor do FIB Bank depõe na CPI da Pandemia s...,A CPI da Pandemia vai ouvir nesta quarta-feira...
5821,6e0e414a-deaf-476a-8dd3-defef8e5f684,Justiça do DF anula prisão em flagrante de ex-...,A 15ª Vara Federal do Distrito Federal declaro...
5822,df9e0959-efed-429b-a72c-83ef827d39d4,"Poupando o erário, Prosus pacifica contrato pa...",Para regularizar a alimentação de pacientes e ...
...,...,...,...
6234,1382b7eb-dc08-4b23-9d48-2b61c5629cc1,MPT cobra multa aplicada à Prefeitura do Rio e...,O Ministério Público do Trabalho (MPT) ajuizo...
6235,b169c93b-61cd-4809-ad60-28220361fa52,OMS: serviços de higiene precários afetam meta...,"A falta de serviços básicos de higiene, como á..."
6236,07a9805a-2be5-4dfc-994c-e3fb2a77d4ce,Operação da PF mira desvios de até R$ 50 milhõ...,A Polícia Federal (PF) realiza a Operação Kic...
6237,1372f91f-cdad-4f70-b769-207c74b01162,PF cumpre mandados de busca e apreensão contra...,A Polícia Federal (PF) e a Controladoria Geral...


# Modelagem de Tópicos

In [25]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
from sklearn.cluster import KMeans
from gensim import corpora, models
from bertopic import BERTopic
#from top2vec import Top2Vec
from tqdm import tqdm

# ====== Função principal ======
def topic_modeling_pipeline(df, text_col="news_content_clean", n_topics=3, n_rounds=5):

    texts = df[text_col].fillna("").astype(str).tolist()
    
    # Vetorização (para métodos baseados em matriz)
    vectorizer = TfidfVectorizer(max_features=5000)
    X = vectorizer.fit_transform(texts)

    # Loop pelos métodos
    for method in ["NMF", "LSA", "BERTopic", "Top2Vec", "HDP", "pLSA"]:
        print(f"\n=== Rodando {method} ===")

        for i in tqdm(range(1, n_rounds + 1)):
            col_name = f"{method}_{i}"

            if method == "NMF":
                model = NMF(n_components=n_topics, random_state=i)
                W = model.fit_transform(X)
                labels = np.argmax(W, axis=1)

            elif method == "LSA":
                model = TruncatedSVD(n_components=n_topics, random_state=i)
                W = model.fit_transform(X)
                labels = np.argmax(W, axis=1)

            elif method == "BERTopic":
                model = BERTopic(language="multilingual", nr_topics="auto", verbose=False)
                topics, _ = model.fit_transform(texts)
                labels = topics

            elif method == "HDP":
                tokenized = [t.split() for t in texts]
                dictionary = corpora.Dictionary(tokenized)
                corpus = [dictionary.doc2bow(t) for t in tokenized]
                hdp = models.hdpmodel.HdpModel(corpus, dictionary)
                # Extrair tópicos mais prováveis
                labels = []
                for doc in corpus:
                    topic_probs = hdp[doc]
                    labels.append(max(topic_probs, key=lambda x: x[1])[0] if topic_probs else -1)

            elif method == "pLSA":
                # pLSA pode ser aproximado usando NMF (sem sparsity)
                model = NMF(n_components=n_topics, random_state=i, init='nndsvd', solver='mu', beta_loss='frobenius', max_iter=300)
                W = model.fit_transform(X)
                labels = np.argmax(W, axis=1)

            # Adiciona resultado ao DataFrame
            df[col_name] = labels

    return df

def topic_modeling_pipeline_long(df, text_col="news_content_clean", n_topics=3, n_rounds=5):

    texts = df[text_col].fillna("").astype(str).tolist()

    # Vetorização (para métodos baseados em matriz)
    vectorizer = TfidfVectorizer(max_features=5000)
    X = vectorizer.fit_transform(texts)

    results = []  # armazenará todos os resultados para concatenação

    methods = ["NMF", "LSA", "LDA", "BERTopic", "HDP", "pLSA"]

    for method in methods:
        print(f"\n=== Rodando {method} ===")

        for i in tqdm(range(1, n_rounds + 1)):
            try:
                if method == "NMF":
                    model = NMF(n_components=n_topics, random_state=i)
                    W = model.fit_transform(X)
                    labels = np.argmax(W, axis=1)

                elif method == "LSA":
                    model = TruncatedSVD(n_components=n_topics, random_state=i)
                    W = model.fit_transform(X)
                    labels = np.argmax(W, axis=1)

                elif method == "LDA":
                    model = LatentDirichletAllocation(n_components=n_topics, random_state=i)
                    W = model.fit_transform(X)
                    labels = np.argmax(W, axis=1)

                elif method == "BERTopic":
                    model = BERTopic(language="multilingual", nr_topics="auto", verbose=False)
                    topics, _ = model.fit_transform(texts)
                    labels = topics

                elif method == "HDP":
                    tokenized = [t.split() for t in texts]
                    dictionary = corpora.Dictionary(tokenized)
                    corpus = [dictionary.doc2bow(t) for t in tokenized]
                    hdp = models.hdpmodel.HdpModel(corpus, dictionary)
                    labels = []
                    for doc in corpus:
                        topic_probs = hdp[doc]
                        labels.append(max(topic_probs, key=lambda x: x[1])[0] if topic_probs else -1)

                elif method == "pLSA":
                    model = NMF(n_components=n_topics, random_state=i, init='nndsvd', solver='mu',
                                beta_loss='frobenius', max_iter=300)
                    W = model.fit_transform(X)
                    labels = np.argmax(W, axis=1)

                else:
                    raise ValueError(f"Método não reconhecido: {method}")

                # monta dataframe parcial
                df_tmp = df.copy()
                df_tmp["method"] = method
                df_tmp["round"] = i
                df_tmp["topic"] = labels
                results.append(df_tmp)

            except Exception as e:
                print(f"[ERRO] {method} rodada {i}: {e}")
                pass

    # concatena tudo num único DataFrame final
    df_final = pd.concat(results, ignore_index=True)
    return df_final

/home/alysson/Área de Trabalho/Alysson/PROCC/00 - Artigos/topic_modelling/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-20 20:47:45.506292: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-20 20:47:45.526915: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761004065.548543   32020 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 0

In [30]:
df_topics_list = []
topics_to_test = [5, 10, 15, 20, 25, 30]
for topic_ in tqdm(topics_to_test, desc="Modelando tópicos..."):
    df_topics = topic_modeling_pipeline_long(df, text_col="news_content_clean", n_topics=topic_, n_rounds=35)
    df_topics.to_parquet(f"../results/df_topics_{topic_}.parquet")

Modelando tópicos...:   0%|                               | 0/6 [00:00<?, ?it/s]


=== Rodando NMF ===



%|                                                    | 0/35 [00:00<?, ?it/s]
%|█▎                                          | 1/35 [00:00<00:07,  4.46it/s]
%|██▌                                         | 2/35 [00:00<00:09,  3.66it/s]
%|███▊                                        | 3/35 [00:00<00:08,  3.83it/s]
%|█████                                       | 4/35 [00:01<00:08,  3.84it/s]
%|██████▎                                     | 5/35 [00:01<00:07,  3.87it/s]
%|███████▌                                    | 6/35 [00:01<00:07,  3.98it/s]
%|████████▊                                   | 7/35 [00:01<00:07,  3.98it/s]
%|██████████                                  | 8/35 [00:02<00:06,  4.02it/s]
%|███████████▎                                | 9/35 [00:02<00:06,  4.04it/s]
%|████████████▎                              | 10/35 [00:02<00:06,  4.11it/s]
%|█████████████▌                             | 11/35 [00:02<00:05,  4.09it/s]
%|██████████████▋                            | 12/35 [00:02<00:


=== Rodando LSA ===



%|                                                    | 0/35 [00:00<?, ?it/s]
%|██▌                                         | 2/35 [00:00<00:02, 15.13it/s]
%|█████                                       | 4/35 [00:00<00:01, 16.49it/s]
%|███████▌                                    | 6/35 [00:00<00:01, 16.31it/s]
%|██████████                                  | 8/35 [00:00<00:01, 15.64it/s]
%|████████████▎                              | 10/35 [00:00<00:01, 14.43it/s]
%|███████████████▉                           | 13/35 [00:00<00:01, 16.65it/s]
%|██████████████████▍                        | 15/35 [00:00<00:01, 17.14it/s]
%|████████████████████▉                      | 17/35 [00:01<00:01, 16.08it/s]
%|████████████████████████▌                  | 20/35 [00:01<00:00, 17.67it/s]
%|███████████████████████████                | 22/35 [00:01<00:00, 16.91it/s]
%|█████████████████████████████▍             | 24/35 [00:01<00:00, 14.68it/s]
%|███████████████████████████████▉           | 26/35 [00:01<00:

...


%|                                                    | 0/35 [00:00<?, ?it/s]
%|█▎                                          | 1/35 [00:33<19:10, 33.83s/it]
%|██▌                                         | 2/35 [01:08<18:51, 34.28s/it]
%|███▊                                        | 3/35 [01:42<18:09, 34.04s/it]
%|█████                                       | 4/35 [02:14<17:18, 33.51s/it]
%|██████▎                                     | 5/35 [02:47<16:31, 33.05s/it]
%|███████▌                                    | 6/35 [03:19<15:47, 32.66s/it]
%|████████▊                                   | 7/35 [03:55<15:48, 33.88s/it]
%|██████████                                  | 8/35 [04:27<15:02, 33.44s/it]
%|███████████▎                                | 9/35 [05:01<14:29, 33.44s/it]
%|████████████▎                              | 10/35 [05:34<13:51, 33.27s/it]
%|█████████████▌                             | 11/35 [06:05<13:05, 32.71s/it]
%|██████████████▋                            | 12/35 [06:38<12:


=== Rodando HDP ===



%|                                                    | 0/35 [00:00<?, ?it/s]
%|█▎                                          | 1/35 [00:05<03:10,  5.61s/it]
%|██▌                                         | 2/35 [00:11<03:08,  5.72s/it]
%|███▊                                        | 3/35 [00:17<03:07,  5.86s/it]
%|█████                                       | 4/35 [00:23<03:04,  5.95s/it]
%|██████▎                                     | 5/35 [00:29<02:56,  5.89s/it]
%|███████▌                                    | 6/35 [00:34<02:48,  5.81s/it]
%|████████▊                                   | 7/35 [00:40<02:40,  5.75s/it]
%|██████████                                  | 8/35 [00:46<02:37,  5.82s/it]
%|███████████▎                                | 9/35 [00:52<02:33,  5.90s/it]
%|████████████▎                              | 10/35 [00:58<02:26,  5.86s/it]
%|█████████████▌                             | 11/35 [01:04<02:21,  5.89s/it]
%|██████████████▋                            | 12/35 [01:10<02:


=== Rodando pLSA ===



%|                                                    | 0/35 [00:00<?, ?it/s]
%|█▎                                          | 1/35 [00:02<01:08,  2.00s/it]
%|██▌                                         | 2/35 [00:03<00:57,  1.73s/it]
%|███▊                                        | 3/35 [00:04<00:50,  1.57s/it]
%|█████                                       | 4/35 [00:06<00:44,  1.43s/it]
%|██████▎                                     | 5/35 [00:07<00:45,  1.51s/it]
%|███████▌                                    | 6/35 [00:09<00:44,  1.52s/it]
%|████████▊                                   | 7/35 [00:10<00:42,  1.53s/it]
%|██████████                                  | 8/35 [00:11<00:36,  1.37s/it]
%|███████████▎                                | 9/35 [00:12<00:32,  1.26s/it]
%|████████████▎                              | 10/35 [00:13<00:29,  1.19s/it]
%|█████████████▌                             | 11/35 [00:15<00:28,  1.18s/it]
%|██████████████▋                            | 12/35 [00:16<00:

In [31]:
df_topics

,id_folder,headline,news_content,news_content_clean,method,round,topic
0,0a2dfc71-c0c5-46ce-96ce-2e56ae73930c,Ministério da Saúde contraria orientação da CG...,Apesar de a Controladoria Geral da União (CGU)...,apes controlador geral uniã cgu ter anunc dia ...,NMF,1,15
1,8f6d7604-046d-4c99-afc3-d7fd8452582c,Clínica clandestina denunciada por torturas e ...,A Superintendência de Vigilância em Saúde de G...,superintendent vigil saúd goiás suvis interdit...,NMF,1,19
2,ddd96e6b-d288-46da-850a-b6f01f1e7958,Diretor do FIB Bank depõe na CPI da Pandemia s...,A CPI da Pandemia vai ouvir nesta quarta-feira...,cpi pandem vai ouv nest quarta-feir 25 part 9h...,NMF,1,17
3,6e0e414a-deaf-476a-8dd3-defef8e5f684,Justiça do DF anula prisão em flagrante de ex-...,A 15ª Vara Federal do Distrito Federal declaro...,15ª var federal distrit federal declar nulidad...,NMF,1,8
4,df9e0959-efed-429b-a72c-83ef827d39d4,"Poupando o erário, Prosus pacifica contrato pa...",Para regularizar a alimentação de pacientes e ...,regulariz aliment pacient profission saúd 1ª p...,NMF,1,1
...,...,...,...,...,...,...,...
88405,1382b7eb-dc08-4b23-9d48-2b61c5629cc1,MPT cobra multa aplicada à Prefeitura do Rio e...,O Ministério Público do Trabalho (MPT) ajuizo...,ministéri públic trabalh mpt ajuiz municípi ri...,pLSA,35,2
88406,b169c93b-61cd-4809-ad60-28220361fa52,OMS: serviços de higiene precários afetam meta...,"A falta de serviços básicos de higiene, como á...",falt servic básic higien águ sabã álcool gel a...,pLSA,35,3
88407,07a9805a-2be5-4dfc-994c-e3fb2a77d4ce,Operação da PF mira desvios de até R$ 50 milhõ...,A Polícia Federal (PF) realiza a Operação Kic...,políc federal pf realiz oper kickback apur pos...,pLSA,35,2
88408,1372f91f-cdad-4f70-b769-207c74b01162,PF cumpre mandados de busca e apreensão contra...,A Polícia Federal (PF) e a Controladoria Geral...,políc federal pf controlador geral uniã cgu re...,pLSA,35,2
